**Keras** is a high-level deep learning API written in Python that is built on top of lower-level frameworks like TensorFlow and Theano. It provides a simple and user-friendly interface to build and train neural networks while abstracting away the complexities of underlying machine learning frameworks.

**Keras Model Types**

Keras provides two main types of models:

Sequential API: Used for building simple, linear stacks of layers.

Functional API: Used for more complex models like multi-input, multi-output, or models with shared layers.

The Sequential API is the easiest way to create a neural network where layers are stacked one after another.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a Sequential model
model = Sequential()

# Add layers
model.add(Dense(64, activation='relu', input_shape=(32,)))  # Input layer + 1st hidden layer
model.add(Dense(32, activation='relu'))  # 2nd hidden layer
model.add(Dense(10, activation='softmax'))  # Output layer

# Summary of the model
model.summary()

n this example, we create a simple feedforward neural network with 2 hidden layers. The Dense layer is a fully connected layer where each neuron is connected to every neuron in the previous layer.

Functional API

The Functional API allows you to define complex models with multiple inputs/outputs or layers that are connected in non-linear ways.

In [ ]:
from tensorflow.keras.layers import Input, Dense

# Define inputs
inputs = Input(shape=(32,))

# Define layers
x = Dense(64, activation='relu')(inputs)  # First hidden layer
x = Dense(32, activation='relu')(x)  # Second hidden layer
outputs = Dense(10, activation='softmax')(x)  # Output layer

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Summary of the model
model.summary()

This is a more flexible way of building neural networks, which is especially useful for architectures like residual networks, where layers are added non-sequentially.

Layers

Layers are the building blocks of neural networks in Keras. They transform input data into output data, often using weights that are updated during training. Some common layers include:

Dense: Fully connected layers.

Conv2D: Convolutional layers for image data.

LSTM: Long Short-Term Memory layers for sequential data.

In [ ]:
from tensorflow.keras.layers import Dense

# A simple Dense layer with 64 units
dense_layer = Dense(64, activation='relu', input_shape=(32,))

In [ ]:
#Example (Conv2D Layer):

from tensorflow.keras.layers import Conv2D

# A 2D convolutional layer with 32 filters and a kernel size of 3x3
conv_layer = Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3))

Compiling the Model

After defining a model, it needs to be compiled. The compilation step specifies:


1.   Loss function: Defines how the model’s performance is evaluated.
2.   Common choices: mean_squared_error for regression, categorical_crossentropy for classification.
3.   Optimizer: Determines how the model's parameters are updated.
Common optimizers: SGD, Adam.
4.   Metrics: Additional measures to track during training and evaluation (e.g., accuracy).






In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Training the Model

The fit() function is used to train the model on a dataset. You need to pass the training data, the target labels, and some training parameters (e.g., batch size and number of epochs).

In [ ]:
import numpy as np

# Dummy data
X_train = np.random.rand(1000, 32)  # 1000 samples, each with 32 features
y_train = np.random.randint(10, size=(1000,))  # 1000 labels (for 10 classes)

# One-hot encoding of labels
y_train = np.eye(10)[y_train]

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

In this example, we use the fit() method to train the model for 10 epochs with a batch size of 32. X_train contains the input features, and y_train contains the one-hot encoded target labels.

Evaluating the Model

After training, you can evaluate the model on a test set using the evaluate() method.

In [ ]:
# Dummy test data
X_test = np.random.rand(200, 32)
y_test = np.random.randint(10, size=(200,))
y_test = np.eye(10)[y_test]

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

Here, evaluate() returns the test loss and accuracy on unseen data.

 Making Predictions

The predict() method is used to generate predictions from the model.

In [ ]:
# Make predictions on new data
X_new = np.random.rand(5, 32)  # 5 new samples
predictions = model.predict(X_new)

print(predictions)

In this case, the model predicts the probabilities for each of the 10 output classes for the 5 new samples.

 Callbacks

Keras provides callbacks, which are functions that can be applied during training to monitor and adjust the training process. Some common callbacks include:

1.  EarlyStopping: Stops training if the model’s performance on the validation set stops improving.
2.  ModelCheckpoint: Saves the model during training.

Example (EarlyStopping and ModelCheckpoint):

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Train the model with callbacks
model.fit(X_train, y_train, epochs=10, batch_size=32,
          validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

Here, the training will stop early if the validation loss doesn’t improve for 3 consecutive epochs, and the best model will be saved to 'best_model.h5'.

 Preprocessing

Keras provides several utilities for preprocessing data. For instance, the ImageDataGenerator class can be used to perform data augmentation on images, which helps to increase the diversity of the training set.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator object for augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)

# Fit the generator on the training data
datagen.fit(X_train)

# Use the augmented data during training
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10)

Transfer Learning

In transfer learning, a pre-trained model (such as VGG16, ResNet) is used as a starting point, and then it is fine-tuned on a new dataset. This is particularly useful when you have a small dataset but want to benefit from a model trained on a large dataset like ImageNet.

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=output)

# Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Here, the convolutional base of the VGG16 model is frozen (i.e., its weights are not updated), and new layers are added for a custom classification task.

Saving and Loading Models

Keras models can be saved and loaded easily using the save() and load_model() functions.

In [ ]:
# Save the entire model to a file
model.save('my_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Load the model from a file
model = load_model('my_model.h5')

# Now the model can be used for inference or further training
predictions = model.predict(X_new)